In [1]:
import warnings
import pprint
import skrebate
import imblearn
from imblearn import under_sampling, over_sampling, combine
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn import (preprocessing, svm, linear_model, ensemble, naive_bayes,
                    tree, neighbors, decomposition, kernel_approximation, cluster)
from sklearn.pipeline import Pipeline
from sklearn.base import clone

from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import (KFold, GroupKFold, StratifiedKFold,
                                    LeaveOneGroupOut, cross_validate,
                                    cross_val_predict, learning_curve,
                                    GridSearchCV)
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel, VarianceThreshold, f_classif
from sklearn.metrics import (r2_score, auc, roc_auc_score, balanced_accuracy_score, 
                             average_precision_score, confusion_matrix, roc_curve,
                             precision_recall_curve)
from sklearn.metrics.scorer import roc_auc_scorer
from sklearn.preprocessing import QuantileTransformer, quantile_transform, StandardScaler, MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
from sklearn.utils.validation import check_memory
from xgboost import XGBRegressor, XGBClassifier
from sklearn.ensemble import RandomForestClassifier

warnings.simplefilter('ignore')

/Users/guq/miniconda3/envs/python3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/Users/guq/miniconda3/envs/python3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
import os
import sys

In [3]:
import numpy as np
import pandas as pd
import re

import plotly.io as pio
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [4]:
work_dir = '/Users/guq/Documents/ohsu/drug_respond/results/RepeatedCV_2019/'
sub1 = 'Hyperparameter_RFC/'
sub2 = 'Hyperparameter_kbest__li_XGBC/'
sub3 = 'Hyperparameter_tree_XGBC/'
#sub3 = 'Hyperparameter Search on collection 18 _iraps/'
#sub4 = 'Hyperparameter Search on collection 19 _xgbregressor'
#sub5 = 'Hyperparameter Search on collection 18 _xgbclassifier_05'

In [5]:
smmart_drugs = ('Afatinib', 'Bortezomib', 'Cabozantinib', 'Dasatinib', 'Erlotinib',
                'Idelalisib', 'Imatinib', 'Olaparib', 'Palbociclib', 'Ponatinib',
                'Rapamycin', 'Sorafenib', 'Sunitinib', 'Trametinib', 'Vorinostat')

def concate_best_result(folder, file_name, scorer, classifier, results, discretize_z_score=-1, exact_z_score=False):
    
    if not file_name.endswith('tabular'):
        return results
    path = os.path.join(folder, file_name)
    res = pd.read_csv(path, sep='\t')
    
    res_filtered = res
    if exact_z_score:
        #print(path)
        if classifier == 'IRAPSClassifier':
            res_filtered = res_filtered[res_filtered['param_estimator__discretize'] == discretize_z_score]
            if discretize_z_score != -1:
                classifier = 'IRAPSClassifier_05'
        else:
            res_filtered = res_filtered[res_filtered['param_estimator__z_score'] == discretize_z_score]
            if discretize_z_score != -1:
                classifier = classifier + '_05'
        #print(classifier)
    res_sort = res_filtered.sort_values(['mean_test_'+scorer, 'std_test_'+scorer], ascending=[False, True])
    res_best = res_sort[['mean_test_'+scorer, 'std_test_'+scorer,'params']].head(1).reset_index(drop=True)
    if not res_best['mean_test_'+scorer].isnull().any():
        if file_name.startswith((smmart_drugs)):
            res_best.insert(loc=0, column='dataset', value=file_name[:-8])
        else:
            res_best.insert(loc=0, column='dataset', value=file_name[:-11])
        res_best.insert(loc=0, column='classifier', value=classifier)
        if results is None:
            results = res_best
        else:
            results = results.append(res_best, ignore_index=True)
    return results

In [6]:
# best AP scores
files1 = os.listdir(work_dir+sub1)
files2 = os.listdir(work_dir+sub2)
files3 = os.listdir(work_dir+sub3)
""""files4 = os.listdir(work_dir+sub4)
files5 = os.listdir(work_dir+sub5)"""
results = None
scorer = 'binarize_average_precision_scorer'
for fl in files1:
    results = concate_best_result(work_dir+sub1, fl, scorer, 'RandomForestClassifier', results)

for fl in files2:
    results = concate_best_result(work_dir+sub2, fl, scorer, 'kbest__li_XGBClassifier', results)
    
for fl in files3:
    results = concate_best_result(work_dir+sub3, fl, scorer, 'tree_XGBClassifier', results)
    
"""for fl in files3:
    results = concate_best_result(work_dir+sub3, fl, scorer, 'IRAPSClassifier', results, discretize_z_score=-1, exact_z_score=True)
for fl in files3:
    results = concate_best_result(work_dir+sub3, fl, scorer, 'IRAPSClassifier', results, discretize_z_score=-0.5, exact_z_score=True)

for fl in files4:
    results = concate_best_result(work_dir+sub4, fl, scorer, 'XGBRegressor', results)"""


results = results.sort_values(['classifier', 'dataset'])
results

,classifier,dataset,mean_test_binarize_average_precision_scorer,std_test_binarize_average_precision_scorer,params
59,RandomForestClassifier,(5Z)-7-Oxozeaenol_1242.tsv,0.497135,0.090452,"{'classifier__class_weight': 'balanced', 'clas..."
191,RandomForestClassifier,5-Fluorouracil_179.tsv,0.558626,0.091371,{'classifier__class_weight': 'balanced_subsamp...
104,RandomForestClassifier,681640_1046.tsv,0.201513,0.060120,"{'classifier__class_weight': None, 'classifier..."
255,RandomForestClassifier,A-443654_86.tsv,0.391934,0.145698,"{'classifier__class_weight': 'balanced', 'clas..."
236,RandomForestClassifier,A-770041_55.tsv,0.402839,0.110030,"{'classifier__class_weight': 'balanced', 'clas..."
143,RandomForestClassifier,AICA Ribonucleotide_1001.tsv,0.506817,0.121539,"{'classifier__class_weight': None, 'classifier..."
25,RandomForestClassifier,AKT inhibitor VIII_171.tsv,0.247819,0.064119,"{'classifier__class_weight': None, 'classifier..."
6,RandomForestClassifier,AKT inhibitor VIII_228.tsv,0.564058,0.089135,"{'classifier__class_weight': 'balanced', 'clas..."
150,RandomForestClassifier,AR-42_272.tsv,0.730871,0.073271,"{'classifier__class_weight': 'balanced', 'clas..."
201,RandomForestClassifier,AS601245_207.tsv,0.227815,0.076535,"{'classifier__class_weight': 'balanced', 'clas..."


In [7]:
results.to_csv(work_dir+'ap.csv', sep=',', index=False)

In [8]:
# best roc-auc scores
files1 = os.listdir(work_dir+sub1)
files2 = os.listdir(work_dir+sub2)
files3 = os.listdir(work_dir+sub3)
"""files4 = os.listdir(work_dir+sub4)
files5 = os.listdir(work_dir+sub5)"""
results_auc = None
scorer = 'binarize_auc_scorer'
for fl in files1:
    results_auc = concate_best_result(work_dir+sub1, fl, scorer, 'RandomForestClassifier', results_auc)

for fl in files2:
    results_auc = concate_best_result(work_dir+sub2, fl, scorer, 'kbest__li_XGBClassifier', results_auc)

for fl in files3:
    results_auc = concate_best_result(work_dir+sub3, fl, scorer, 'tree_XGBClassifier', results_auc)
    
"""for fl in files3:
    results_auc = concate_best_result(work_dir+sub3, fl, scorer, 'IRAPSClassifier', results_auc, exact_z_score=True)
for fl in files3:
    results_auc = concate_best_result(work_dir+sub3, fl, scorer, 'IRAPSClassifier', results_auc, discretize_z_score=-0.5, exact_z_score=True)

for fl in files4:
    results_auc = concate_best_result(work_dir+sub4, fl, scorer, 'XGBRegressor', results_auc)"""

results_auc = results_auc.sort_values(['classifier', 'dataset'])
results_auc

,classifier,dataset,mean_test_binarize_auc_scorer,std_test_binarize_auc_scorer,params
59,RandomForestClassifier,(5Z)-7-Oxozeaenol_1242.tsv,0.773172,0.057460,{'classifier__class_weight': 'balanced_subsamp...
191,RandomForestClassifier,5-Fluorouracil_179.tsv,0.830036,0.041431,{'classifier__class_weight': 'balanced_subsamp...
104,RandomForestClassifier,681640_1046.tsv,0.576178,0.066405,{'classifier__class_weight': 'balanced_subsamp...
255,RandomForestClassifier,A-443654_86.tsv,0.659055,0.092668,{'classifier__class_weight': 'balanced_subsamp...
236,RandomForestClassifier,A-770041_55.tsv,0.740500,0.082914,"{'classifier__class_weight': 'balanced', 'clas..."
143,RandomForestClassifier,AICA Ribonucleotide_1001.tsv,0.794760,0.061862,"{'classifier__class_weight': None, 'classifier..."
25,RandomForestClassifier,AKT inhibitor VIII_171.tsv,0.634784,0.048619,"{'classifier__class_weight': 'balanced', 'clas..."
6,RandomForestClassifier,AKT inhibitor VIII_228.tsv,0.839149,0.043347,"{'classifier__class_weight': 'balanced', 'clas..."
150,RandomForestClassifier,AR-42_272.tsv,0.880167,0.031704,{'classifier__class_weight': 'balanced_subsamp...
201,RandomForestClassifier,AS601245_207.tsv,0.615308,0.061504,"{'classifier__class_weight': 'balanced', 'clas..."


In [9]:
results_auc.to_csv(work_dir+'roc-auc.csv', sep=',', index=False)

In [12]:
data1 = go.Bar(
    x = results[results['classifier'] == 'RandomForestClassifier']['dataset'],
    y = results[results['classifier'] == 'RandomForestClassifier']['mean_test_binarize_average_precision_scorer'],
    name = 'RFC_ap'
)

data2 = go.Bar(
    x = results[results['classifier'] == 'kbest__li_XGBClassifier']['dataset'],
    y = results[results['classifier'] == 'kbest__li_XGBClassifier']['mean_test_binarize_average_precision_scorer'],
    name = 'kbest__li_XGBC_ap'
)

data3 = go.Bar(
    x = results[results['classifier'] == 'tree_XGBClassifier']['dataset'],
    y = results[results['classifier'] == 'tree_XGBClassifier']['mean_test_binarize_average_precision_scorer'],
    name = 'tree_XGBC_ap'
)

data11 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'RandomForestClassifier']['dataset'],
    y = results_auc[results_auc['classifier'] == 'RandomForestClassifier']['mean_test_binarize_auc_scorer'],
    name = 'RFC_auc'
)

data12 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'kbest__li_XGBClassifier']['dataset'],
    y = results_auc[results_auc['classifier'] == 'kbest__li_XGBClassifier']['mean_test_binarize_auc_scorer'],
    name = 'kbest__li_XGBC_auc'
)

data13 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'tree_XGBClassifier']['dataset'],
    y = results_auc[results_auc['classifier'] == 'tree_XGBClassifier']['mean_test_binarize_auc_scorer'],
    name = 'tree_XGBC_auc'
)

layout = go.Layout(
    xaxis=dict(
        title='Dataset'
    ),
    yaxis=dict(
        title='Performance score'
    ),
    barmode = 'group'
)
fig = go.Figure(data=[data1, data2, data3], layout=layout)
iplot(fig)
#iplot(fig, filename='/Users/guq/Downloads/newplot_ap_1.png', image='png')


fig = go.Figure(data=[data11, data12, data13], layout=layout)
#iplot(fig, filename='/Users/guq/Downloads/newplot_roc_1.png', image='png')
iplot(fig)
# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/

In [13]:
trace1 = {
    "type": 'violin',
    "x": results['classifier'],
    "y": results['mean_test_binarize_average_precision_scorer'],
    "legendgroup": 'AP',
    "scalegroup": 'AP',
    "name": 'AP',
    "box": {
        "visible": True
    },
    "meanline": {
        "visible": True
    },
    "line": {
        "color": 'blue'
    }
}

trace2 = {
    "type": 'violin',
    "x": results_auc['classifier'],
    "y": results_auc['mean_test_binarize_auc_scorer'],
    "legendgroup": 'ROC-AUC',
    "scalegroup": 'ROC-AUC',
    "name": 'ROC-AUC',
    "box": {
        "visible": True
    },
    "meanline": {
        "visible": True
    },
    "line": {
        "color": 'pink'
    }
}

layout = {
    "yaxis": {
        "zeroline": False,
    },
    "violinmode": 'group'
}
fig = go.Figure(data=[trace1, trace2], layout=layout)
#iplot(fig, filename='/Users/guq/Downloads/newplot_violin_smmart.png', image='png')
iplot(fig)
# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/